In [1]:
import json
import torch
import torch.nn as nn
import bitsandbytes as bnb
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model
from fuzzywuzzy import fuzz
import re

/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cache_dir = "/cs/student/projects1/aibh/2024/tpatil/.cache/huggingface/"
base_model_name = "microsoft/Phi-3.5-mini-instruct"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_8bit=True,
    device_map='auto',
    trust_remote_code=True,
    cache_dir=cache_dir
    )

base_model_tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True,
    cache_dir=cache_dir
    )

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [02:12<00:00, 66.19s/it]


## Data Pre-processing

In [4]:
def load_finqa_dataset(path):
    with open(path, "r") as f:
        return json.load(f)
    
train_data_finqa = load_finqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/FinQA/train.json")
val_data_finqa = load_finqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/FinQA/dev.json")
test_data_finqa = load_finqa_dataset("/cs/student/projects1/aibh/2024/tpatil/comp0087/FrugalML/datasets/FinQA/test.json")

In [5]:
def create_prompt_instance(table, pre_text, post_text, question, answer, program_reasoning=None):
    set_context = "You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\n"
    table_prompt = "The table provide the financial data. All the elements in the table are separated by \"|\". The first row of the table contains the column names. In the following rows, the first column contains the row name and the rest of the elements are the values in the row assigned to the respective columns. Interpret the table and use the data in it to calculate the answer to the provided quesions.\n"
    pre_text_prompt = "The pre_text provides some context about the data before the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each pre_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n"
    post_text_prompt = "The post_text provides some context about the data after the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each post_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n"
    question_prompt = "The question is asked based on the data in the table and the paragraph. You are expected to answer the question based on the data in the table and the paragraph.\n"
    answer_prompt = "" #"You are expected to answer the question based on the data in the table and the paragraph. Provide only the answer to the question and do not repeat the question. Use the answers provided as labels to learn the correct way to answer the question.\n"
    # program_reasoning_prompt = "The program reasoning provides the steps to calculate the answer to the question. You can learn how to use the program_reasoning to calculate the answer to the question.\n"
    answer_instruction_prompt = "\nInstruction: Answer the question based on the data in the table and the paragraph. Provide only the answer to the question and do not repeat the question. Use the answers provided as labels to learn the correct way to answer the question.\nAnswer: \n"

    table_prompt += f"\nTable:\n"
    for row in table:
        table_prompt += "|".join([str(cell) for cell in row]) + " \n"

    pre_text_prompt += f"\nPre_text:\n {pre_text}"
    post_text_prompt += f"\nPost_text:\n {post_text}"
    question_prompt += f"\nQuestion:\n {question}"
    answer_prompt += f"\nAnswer:\n {answer}"
    # program_reasoning_prompt += f"\nDerivation:\n {derivation}"

    return set_context, table_prompt, pre_text_prompt, post_text_prompt, question_prompt, answer_prompt, answer_instruction_prompt #, program_reasoning_prompt

In [6]:
class FinQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.qa_pairs = []

    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        tables = item.get('table', [])
        pre_text = item.get('pre_text', [])
        post_text = item.get('post_text', [])
        qa = item.get('qa', {})

        question = qa["question"].strip()
        answer = qa["answer"]

        set_context, table_prompt, pre_text_prompt, post_text_prompt, question_prompt, answer_prompt, answer_instruction_prompt = create_prompt_instance(tables, pre_text, post_text, question, answer)
        input_context = (set_context + pre_text_prompt + table_prompt + post_text_prompt + question_prompt + answer_instruction_prompt).strip()
        label_text = answer_prompt.strip()

        inputs = self.tokenizer( 
                    input_context, 
                    # max_length=self.max_length, 
                    # truncation=True, 
                    # padding="max_length", 
                    return_tensors="pt"
                )

        labels = self.tokenizer(
                    label_text, 
                    # max_length=self.max_length, 
                    # truncation=True, 
                    # padding="max_length", 
                    return_tensors="pt"
                )

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        labels = labels["input_ids"].squeeze()


        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
            # "input_context": input_context,
            # "label_text": label_text
        }

In [7]:
train_dataset_finqa = FinQADataset(train_data_finqa, base_model_tokenizer)
val_dataset_finqa = FinQADataset(val_data_finqa, base_model_tokenizer)
test_dataset_finqa = FinQADataset(test_data_finqa, base_model_tokenizer)

train_dataloader_finqa = DataLoader(
    train_dataset_finqa, 
    batch_size=1, 
    shuffle=True
)

val_dataloader_finqa = DataLoader(
    val_dataset_finqa, 
    batch_size=1, 
    shuffle=False
)

test_dataloader_finqa = DataLoader(
    test_dataset_finqa, 
    batch_size=1, 
    shuffle=False
)

In [8]:
next(iter(train_dataloader_finqa))

{'input_ids': tensor([[  887,   526,   385,  ...,    13, 22550, 29901]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]),
 'labels': tensor([[  673, 29901,    13, 29871, 29929, 29947, 29889, 29947, 29995]])}

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Inference on the base model

In [ ]:
# def model_inference(model, test_dataloader, tokenizer, target_count=20):
base_model.eval()

generated_answers = []
reference_answers = []
reference_questions = []
sample_count = 0
target_count = 20

with torch.no_grad():
    for batch in test_dataloader_finqa:
        if sample_count >= target_count:
            break

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        batch_size = input_ids.shape[0]
        
        # Determine how many samples to take from this batch
        samples_needed = min(batch_size, target_count - sample_count)
        
        # Take only the needed samples from this batch
        batch_input_ids = input_ids[:samples_needed]
        batch_attention_mask = attention_mask[:samples_needed]
        batch_labels = labels[:samples_needed]
        
        # Decode questions
        batch_questions = base_model_tokenizer.batch_decode(batch_input_ids, skip_special_tokens=True)
        # Generate answers
        with torch.cuda.amp.autocast(enabled=True):
            outputs = base_model.generate(
                input_ids=batch_input_ids,
                attention_mask=batch_attention_mask,
                max_new_tokens=128,
                # do_sample=False,
                num_beams=4,
                # top_k=10,
                # early_stopping=True,
                # pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )
        
        # Decode generated answers
        batch_answers = base_model_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for batch_answer in batch_answers:
            if ("Answer:" in batch_answer):
                batch_answer = re.search(r'(?<=Answer:)\s?.+?(?=\\n|\n|$)', batch_answer).group(0)
            generated_answers.append(batch_answer)
        
        # Add to our collections
        reference_questions.append(batch_questions)
        reference_answer = base_model_tokenizer.batch_decode(batch_labels, skip_special_tokens=True)
        reference_answer = re.search(r'(?<=Answer:\\n\s).+(?=(\"|\')\])', str(reference_answer))
        if reference_answer:
            reference_answer = reference_answer.group(0)
        else:
            reference_answer = ""
        # list_element = re.search('(?<=\[).+(?=\])', str(reference_answer))
        # if list_element:
        #     reference_answer = list_element.group(0).replace(", ", "; ").replace("'", "")
        reference_answers.append(reference_answer)
        
        # Update count
        sample_count += samples_needed

print(f"Collected {len(reference_questions)} samples:")
for i, (question, answer) in enumerate(zip(reference_questions, generated_answers)):
    print(f"\nSample {i+1}:")
    print(f"Q: {question}")
    print(f"A: {answer}")


/tmp/ipykernel_222181/1435126505.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Collected 20 samples:

Sample 1:
Q: ['You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe pre_text provides some context about the data before the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each pre_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n\nPre_text:\n [\'entergy corporation and subsidiaries management 2019s financial discussion and analysis a result of the entergy louisiana and entergy gulf states louisiana business combination , results of operations for 2015 a

In [ ]:
# model_inference(base_model, test_dataloader_finqa, base_model_tokenizer, target_count=20)

In [ ]:
generated_answers

['$84 million',
 '8.1%',
 'Approximately 12.7% increase',
 '121.4 million',
 '22.57%',
 'Approximately 8.7% increase',
 'The table does not provide the percentage of total recourse debt that is current. Therefore, based on the given data, it is not possible to determine the percentage of total recourse debt that is current. ',
 'Approximately 11.6%',
 'yes',
 '$592 million decrease',
 'The growth rate is $1.04 billion - $492.6 million = $517.4 million. To find the growth rate, subtract the 2006 value from the 2007 value: $1.04 billion - $492.6 million = $517.4 million. The growth rate in the balance of standby letters of credit from 2006 to 2007 is $517.4 million.',
 'Working capital divided by total assets',
 '$ 137.4 million',
 'yes',
 'The percentage change in deferred tax assets from 2013 to 2014 is ((195237 - 177947) / 177947) * 100 = 9.79%',
 '(4648 + 3625 + 1620) / 3 = 3367.6666666666665',
 'Approximately 65 destinations per continent',
 '($705.4 million + $703.1 million) / 2 = 

In [ ]:
reference_answers

['94',
 '14%',
 '9.9%',
 '2.9%',
 '',
 '7%',
 '10%',
 '12%',
 'yes',
 '-35',
 '-4.4%',
 '22.4%',
 '24.6%',
 'no',
 '1.7%',
 '3298',
 '65',
 '704.25',
 '11.0%',
 '68.9']

In [21]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def clean_answer(text):
    """Extract and format numerical answer"""
#    if ':' in text:
#        text = text.split(':')[-1]

    # Handle percentages first
    # percent_match = re.search(r'[-+]?\d*\.?\d+\s*%?', text)
    percent_match = re.search(r'[-+]?\d*\.?\d+\s*%', text)
    if percent_match:
        number = float(percent_match.group(0).replace('%', '').strip())
        # If the number is small (likely decimal), convert to percentage
#        if abs(number) < 1:
#            number *= 100
        # Round to one decimal place and add % symbol
        return f"{round(number, 0)}%"
        # return str(number)

    # Handle regular numbers
    pre_decimal_match = text
    if '=' in text:
        pre_decimal_match = re.search(r'(?<=\=\s).+', text, re.MULTILINE).group(0)
    decimal_match = re.search(r'[-+]?\d*,?\.?\d+', pre_decimal_match, re.MULTILINE)
    # decimal_match = re.search(r'(?<=\$)(\d{1,3}(,\d{3})*(\.\d+)?|\d+(\.\d+)?)', text)
    # if decimal_match:
    #     match_str = decimal_match.group(0)
    #     if is_number(match_str):
    #         number = float(match_str)
    #         if abs(round(number) - number) < 0.01:
    #             return str(round(number))
    #     else:
    #         number = match_str
    #     # If it's close to an integer, round it
    #     # Otherwise, round to one decimal place
    #     return str(number)
    if decimal_match:
        number = float(decimal_match.group(0))
        # If it's close to an integer, round it
        if abs(round(number) - number) < 0.01:
            return str(round(number))
        # Otherwise, round to one decimal place
        return str(round(number, 0))

    # Handle yes/no answers
    text = text.lower().strip()
    if 'yes' in text or 'true' in text:
        return 'yes'
    if 'no' in text or 'false' in text:
        return 'no'

    return text.strip()

In [18]:
total_samples = 0
correct_predictions = 0
threshold = 85  # Minimum similarity percentage for correct match
similarity_scores = []
SETTING = "FinQA"

# import time

# start = time.time()

for i, generated_answer in enumerate(generated_answers):
    total_samples += 1
    clean_p = clean_answer(generated_answer)
    reference_answer = reference_answers[i]
    clean_e = clean_answer(reference_answer)


    # Compute similarity score
    # similarity = fuzz.ratio(predicted_answer.lower(), expected_answer.lower())
    similarity = fuzz.ratio(clean_p.lower(), clean_e.lower())
    similarity_scores.append(similarity)
    print()
    print()
    print("************************************************************************")
    print (f"*** Input {total_samples} *** ")
#    print(example["input_text"])
    print ("*** Expected answer *** ")
    print(reference_answer)
    print ("*** Clean Expected answer *** ")
    print(clean_e)
    print ("*** Predicted answer *** ")
    print(generated_answer)
    print ("*** Clean Predicted answer *** ")
    print(clean_p)

    # Count as correct if similarity is above threshold
    if similarity >= threshold:
        correct_predictions += 1

# Print Results
accuracy = correct_predictions / total_samples * 100
avg_similarity = sum(similarity_scores) / total_samples

print(f"*************************************{SETTING}***********************************")

print(f"Total Samples: {total_samples}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Average Similarity Score: {avg_similarity:.2f}%")

# Block of code to time
# end = time.time()

# print("Elapsed time:", end - start, "seconds")
# print("Avg Inf time:", (end - start)/dataset.shape[0], "seconds")



************************************************************************
*** Input 1 *** 
*** Expected answer *** 
94
*** Clean Expected answer *** 
94
*** Predicted answer *** 
$84 million
*** Clean Predicted answer *** 
84


************************************************************************
*** Input 2 *** 
*** Expected answer *** 
14%
*** Clean Expected answer *** 
14.0%
*** Predicted answer *** 
8.1%
*** Clean Predicted answer *** 
8.0%


************************************************************************
*** Input 3 *** 
*** Expected answer *** 
9.9%
*** Clean Expected answer *** 
10.0%
*** Predicted answer *** 
Approximately 12.7% increase
*** Clean Predicted answer *** 
13.0%


************************************************************************
*** Input 4 *** 
*** Expected answer *** 
2.9%
*** Clean Expected answer *** 
3.0%
*** Predicted answer *** 
121.4 million
*** Clean Predicted answer *** 
121.0


*********************************************************

## Model Pre-training using LoRA

In [3]:
for param in base_model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

base_model.gradient_checkpointing_enable()
base_model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)
base_model.lm_head = CastOutputToFloat(base_model.lm_head)

In [4]:
def print_trainable_parameters(base_model):
    trainable_params = 0
    all_params = 0
    for _, param in base_model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable parameters: {trainable_params} || All parameters: {all_params} || Trainable parameters %: {trainable_params/all_params*100}")

In [6]:
# Inspect the named modules in the model
for name, module in base_model.named_modules():
    print(name)


model
model.embed_tokens
model.embed_dropout
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.qkv_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.activation_fn
model.layers.0.input_layernorm
model.layers.0.resid_attn_dropout
model.layers.0.resid_mlp_dropout
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.qkv_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.activation_fn
model.layers.1.input_layernorm
model.layers.1.resid_attn_dropout
model.layers.1.resid_mlp_dropout
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.o_proj
model.layers.2.self_attn.qkv_proj
model.layers.2.self_attn.rotary_emb
model.layers.

In [11]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.2,
    # target_modules="all-linear",
    target_modules=['qkv_proj', 'o_proj', 'gate_up_proj', 'down_proj'],
    bias="none",
    task_type="CAUSAL_LM"
)

base_model = get_peft_model(base_model, config)
print_trainable_parameters(base_model)

Trainable parameters: 25165824 || All parameters: 3846245376 || Trainable parameters %: 0.6542958532243159


In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     # evaluation_strategy="steps",
#     # eval_steps=500,
#     # save_steps=500,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     num_train_epochs=5,
#     # gradient_accumulation_steps=1,
#     warmup_steps=100,
#     max_steps=1000,
#     # weight_decay=0.01,
#     learning_rate=5e-3,
#     logging_dir="./logs",
#     logging_steps=1,
#     fp16=True,
#     report_to="none",
#     load_best_model_at_end=True,
#     label_names=["labels"],
#     # metric_for_best_model="accuracy"
# )

training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="steps",
    # eval_steps=500,
    # save_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    # gradient_accumulation_steps=1,
    warmup_steps=100,
    max_steps=1000,
    # num_train_epochs=5,
    # eval_strategy="epoch",
    # save_strategy="epoch",
    weight_decay=0.01,
    learning_rate=2e-4,
    logging_dir="./logs",
    logging_steps=1,
    fp16=True,
    report_to="none",
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=base_model,
    train_dataset=train_dataset_finqa,
    eval_dataset=val_dataset_finqa,
    tokenizer=base_model_tokenizer,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=base_model_tokenizer, mlm=False)
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_39216/2797023835.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
base_model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.2, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_mag

In [14]:
trainer.train()
trainer.save_model("./results_finqa/finqa_model_v6")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


You are not running the flash-attention implementation, expect numerical differences.
/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.609800
2,1.642300
3,1.384000
4,1.630600
5,1.830800
6,1.978100
7,1.761400
8,1.748400
9,1.615400
10,1.694800


/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [15]:
trainer.evaluate()

/cs/student/projects1/aibh/2024/tpatil/venv_comp0188/lib64/python3.9/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'eval_loss': 0.8875550031661987,
 'eval_runtime': 292.3843,
 'eval_samples_per_second': 3.02,
 'eval_steps_per_second': 3.02,
 'epoch': 0.3199488081906895}

In [ ]:
ft_model_path = "./results_finqa/finqa_model_v5"
lora_config = PeftConfig.from_pretrained(ft_model_path)
ft_model = PeftModel.from_pretrained(
    base_model, 
    ft_model_path, 
    config=lora_config,
    load_in_8bit=True,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16
)
# ft_model = ft_model.to(device)

In [19]:
# def model_inference(model, test_dataloader, tokenizer, target_count=20):
ft_model.eval()

lora_generated_answers = []
lora_reference_answers = []
lora_reference_questions = []
sample_count = 0
target_count = 20

with torch.no_grad():
    for batch in test_dataloader_finqa:
        if sample_count >= target_count:
            break

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        batch_size = input_ids.shape[0]
        
        # Determine how many samples to take from this batch
        samples_needed = min(batch_size, target_count - sample_count)
        
        # Take only the needed samples from this batch
        batch_input_ids = input_ids[:samples_needed]
        batch_attention_mask = attention_mask[:samples_needed]
        batch_labels = labels[:samples_needed]
        
        # Decode questions
        batch_questions = base_model_tokenizer.batch_decode(batch_input_ids, skip_special_tokens=True)
        # Generate answers
        with torch.cuda.amp.autocast(enabled=True):
            outputs = ft_model.generate(
                input_ids=batch_input_ids,
                attention_mask=batch_attention_mask,
                max_new_tokens=128,
                # do_sample=False,
                num_beams=4,
                # top_k=10,
                # early_stopping=True,
                # pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
            )
        
        # Decode generated answers
        batch_answers = base_model_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for batch_answer in batch_answers:
            if ("Answer:" in batch_answer):
                batch_answer = re.search(r'(?<=Answer:)\s?.+?(?=\\n|\n|$)', batch_answer).group(0)
            lora_generated_answers.append(batch_answer.strip())
        
        # Add to our collections
        lora_reference_questions.append(batch_questions)
        lora_reference_answer = base_model_tokenizer.batch_decode(batch_labels, skip_special_tokens=True)
        lora_reference_answer = re.search(r'(?<=Answer:\\n\s).+(?=(\"|\')\])', str(lora_reference_answer))
        if lora_reference_answer:
            lora_reference_answer = lora_reference_answer.group(0)
        else:
            lora_reference_answer = ""
        # list_element = re.search('(?<=\[).+(?=\])', str(reference_answer))
        # if list_element:
        #     reference_answer = list_element.group(0).replace(", ", "; ").replace("'", "")
        lora_reference_answers.append(lora_reference_answer)
        
        # Update count
        sample_count += samples_needed

print(f"Collected {len(lora_reference_questions)} samples:")
for i, (question, answer) in enumerate(zip(lora_reference_questions, lora_generated_answers)):
    print(f"\nSample {i+1}:")
    print(f"Q: {question}")
    print(f"A: {answer}")


/tmp/ipykernel_52019/3864758609.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Collected 20 samples:

Sample 1:
Q: ['You are an intelligent financial data analyst. You are given a table with financial data. You are also given a paragraph that provides some context about the data in the table. You are asked a question about the data in the table or paragraph. You are expected to answer the question based on the data in the table and the paragraph.\nThe pre_text provides some context about the data before the table. It may contain information that is not present in the table. It may also contain some numbers which might require arithmatic processing to get the answer. There may be multiple sentences separated by comma (,). Interpret each pre_text paragraph and use the data and description in it to infer the answer to the provided quesions.\n\nPre_text:\n [\'entergy corporation and subsidiaries management 2019s financial discussion and analysis a result of the entergy louisiana and entergy gulf states louisiana business combination , results of operations for 2015 a

In [22]:
total_samples = 0
correct_predictions = 0
threshold = 85  # Minimum similarity percentage for correct match
similarity_scores = []
SETTING = "FinQA"

# import time

# start = time.time()

for i, lora_generated_answer in enumerate(lora_generated_answers):
    total_samples += 1
    clean_p = clean_answer(lora_generated_answer)
    lora_reference_answer = lora_reference_answers[i]
    clean_e = clean_answer(lora_reference_answer)


    # Compute similarity score
    # similarity = fuzz.ratio(predicted_answer.lower(), expected_answer.lower())
    similarity = fuzz.ratio(clean_p.lower(), clean_e.lower())
    similarity_scores.append(similarity)
    print()
    print()
    print("************************************************************************")
    print (f"*** Input {total_samples} *** ")
#    print(example["input_text"])
    print ("*** Expected answer *** ")
    print(lora_reference_answer)
    print ("*** Clean Expected answer *** ")
    print(clean_e)
    print ("*** Predicted answer *** ")
    print(lora_generated_answer)
    print ("*** Clean Predicted answer *** ")
    print(clean_p)

    # Count as correct if similarity is above threshold
    if similarity >= threshold:
        correct_predictions += 1

# Print Results
accuracy = correct_predictions / total_samples * 100
avg_similarity = sum(similarity_scores) / total_samples

print(f"*************************************{SETTING}***********************************")

print(f"Total Samples: {total_samples}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Average Similarity Score: {avg_similarity:.2f}%")

# Block of code to time
# end = time.time()

# print("Elapsed time:", end - start, "seconds")
# print("Avg Inf time:", (end - start)/dataset.shape[0], "seconds")



************************************************************************
*** Input 1 *** 
*** Expected answer *** 
94
*** Clean Expected answer *** 
94
*** Predicted answer *** 
$ 5829 - $ 5735 = $ 94
*** Clean Predicted answer *** 
94


************************************************************************
*** Input 2 *** 
*** Expected answer *** 
14%
*** Clean Expected answer *** 
14.0%
*** Predicted answer *** 
2.3% ( 2.3 % )
*** Clean Predicted answer *** 
2.0%


************************************************************************
*** Input 3 *** 
*** Expected answer *** 
9.9%
*** Clean Expected answer *** 
10.0%
*** Predicted answer *** 
( 153.7 - 139.9 ) / 139.9 * 100% = 10%
*** Clean Predicted answer *** 
100.0%


************************************************************************
*** Input 4 *** 
*** Expected answer *** 
2.9%
*** Clean Expected answer *** 
3.0%
*** Predicted answer *** 
121.4
*** Clean Predicted answer *** 
121.0


*********************************